# Реализация базовых метрик

Ваша задача реализовать основные метрики которые мы сегодня обсудили и сравнить их на игрушечном примере.

In [64]:
UserRatings = {\
               "User1": [5,3,5,2,1,5,1,5,4,4],\
               "User2": [3,3,3,2,2,3,3,5,4,5],\
               "User3": [1,5,5,1,1,3,3,3,4,5]\
              }

In [65]:
Model1Ratings = {\
               "User1": [1,3,4,2,2,4,3,4,4,4],\
               "User2": [1,3,4,2,1,3,2,5,5,5],\
               "User3": [1,3,5,1,3,4,4,5,4,3]\
              }

In [66]:
Model2Ratings = {\
               "User1": [5,4,5,4,2,3,2,5,5,5],\
               "User2": [4,4,3,3,1,2,2,3,3,5],\
               "User3": [3,5,5,2,4,2,2,4,4,4]\
              }

Выше вам даны оценки которые пользователи поставили фильмам и оценки которые предложили две модели. 

*Задача 1:* Вычислите MAE, MSE и RMSE для этих моделей. Какая модель лучше?

*Задача 2:* Реализуйте функцию Recomend(MarkList, N) которая по списку оценок выводит лист из N id фильмов, соответствующих лучшим оценкам. При равенстве оценок лучше считается вильм с меньшим id. id фильма соответствует индекcу в MarkList. 

*Задача 3:* Для каждого пользователя и для каждой модели постройте рекомендацию 5 лучших фильмов, используя функцию  Recomend(MarkList, N). Считая настоящие(UserRatings) оценки положительными в случае 5 или 4 и отрицательными во всех других случаях, посчитайте precision, recall и roc_auc.

*Задача 4:* Посчитайте NDCG для построенной выше рекомендации с бинарными оценками.

*Задача 5:* Посчитайте NDCG для построенной выше рекомендации, считая что $r_i=2$, при оценке 5, $r_i=1$, при оценке 4, $r_i=0$, в остальных случаях.  

*Задача 6:* Сравните модели по всем метрикам.


In [67]:
def MAE(RealRatings, ModelRatings):
    summa = 0
    cnt = 0
    for usr in ModelRatings:
        for i in range(len(ModelRatings[usr])):
            summa += abs(ModelRatings[usr][i] - RealRatings[usr][i])
            cnt += 1
    return summa / cnt

In [68]:
def MSE(RealRatings, ModelRatings):
    summa = 0
    cnt = 0
    for usr in ModelRatings:
        for i in range(len(ModelRatings[usr])):
            summa += (ModelRatings[usr][i] - RealRatings[usr][i])**2
            cnt += 1
    return summa / cnt

In [69]:
MAE(UserRatings, Model1Ratings)

0.8666666666666667

In [70]:
MAE(UserRatings, Model2Ratings)

0.9333333333333333

In [71]:
MSE(UserRatings, Model1Ratings)

1.6666666666666667

In [72]:
MSE(UserRatings, Model2Ratings)

1.4

In [73]:
def Recomend(MarkList, N):
    order = [x for x in range(len(MarkList))]
    return sorted(order, reverse = True, key = lambda x: (MarkList[x], -x))[:N]

In [74]:
Recomend(Mode1Ratings["User1"], 100)

[2, 5, 7, 8, 9, 1, 6, 3, 4, 0]

In [75]:
N = 2
good = 0
cnt = 0
for u in UserRatings:
    res = Recomend(Mode1Ratings[u], N)
    for i in res:
        if UserRatings[u][i]>3: good += 1
        cnt += 1

recall = good/cnt

TotalGood = 0

for u in UserRatings:
    for i in UserRatings[u]:
        if i>3: TotalGood += 1

precision = good/TotalGood

print(precision, recall)

0.38461538461538464 0.8333333333333334


In [94]:
from math import log2
def NDCGk(real_values, model_order, rfunc, k):
    ideal_score = sorted([rfunc(x) for x in real_values], reverse = True)
    model_score = [rfunc(real_values[x]) for x in model_order] 
    #print(ideal_score)
    #print(model_score)
    DCG = 0
    iDCG = 0
    for i in range(k):
        DCG += 2**model_score[i]/log2(i+2)
    for i in range(k):
        iDCG += 2**ideal_score[i]/log2(i+2)
    return (DCG/iDCG)

In [87]:
def more3(x):
    if x > 3: 
        return 1
    else:
        return 0

In [88]:
def ver2(x):
    if x == 5:
        return 2
    elif x == 4:
        return 1
    else:
        return 0
    

In [90]:
NDCGk(UserRatings["User1"], Recomend(Model1Ratings["User1"], 5),ver2, 5)

[2, 2, 2, 2, 1, 1, 0, 0, 0, 0]
[2, 2, 2, 1, 1]


0.9218382134470557

In [102]:
sum1 = 0
sum2 = 0
N = 10
for u in Model1Ratings:
    sum1 += NDCGk(UserRatings[u], Recomend(Model1Ratings[u], N),ver2, N)
    sum2 += NDCGk(UserRatings[u], Recomend(Model2Ratings[u], N),ver2, N)
print(sum1/3, sum2/3)

0.9405241729242503 0.9458572302014999
